### Download Ollama from here (https://ollama.com/download/mac)
### Download any model from the ollama (ex gpt-oss:20b) -> ollama pull gpt-oss:20b     
### Execute -> pip3 install langchain_ollama
### Execute -> pip3 install langgraph
### Execute -> pip3 install langchain
### Follow the ollama langchain guide (https://docs.langchain.com/oss/python/integrations/chat/ollama)
    - https://docs.langchain.com/oss/python/langgraph/quickstart#full-code-example


In [18]:
from langchain_ollama import ChatOllama
llm = ChatOllama(model="llama3.1:8b")
llm.invoke("Hello, How are you?")


AIMessage(content="I'm just a computer program, so I don't have feelings like humans do. But thank you for asking! I'm functioning properly and ready to help with any questions or tasks you may have. How about you? How's your day going so far?", additional_kwargs={}, response_metadata={'model': 'llama3.1:8b', 'created_at': '2025-12-18T09:59:59.19203Z', 'done': True, 'done_reason': 'stop', 'total_duration': 1949466334, 'load_duration': 110700875, 'prompt_eval_count': 16, 'prompt_eval_duration': 270559417, 'eval_count': 53, 'eval_duration': 1165901701, 'logprobs': None, 'model_name': 'llama3.1:8b', 'model_provider': 'ollama'}, id='lc_run--019b30e6-e638-7fa3-bb79-e44fbf1ad3ae-0', usage_metadata={'input_tokens': 16, 'output_tokens': 53, 'total_tokens': 69})

In [19]:
# Step 1: Define tools and model

from langchain.chat_models import init_chat_model
from langchain.tools import tool


# Define tools
@tool
def multiply(a: int, b: int) -> int:
    """Multiply `a` and `b`.

    Args:
        a: First int
        b: Second int
    """
    result = a * b
    print(f"  🔧 multiply({a}, {b}) = {result}")
    return result


@tool
def add(a: int, b: int) -> int:
    """Adds `a` and `b`.

    Args:
        a: First int
        b: Second int
    """
    result = a + b
    print(f"  🔧 add({a}, {b}) = {result}")
    return result


@tool
def divide(a: int, b: int) -> float:
    """Divide `a` and `b`.

    Args:
        a: First int
        b: Second int
    """
    result = a / b
    print(f"  🔧 divide({a}, {b}) = {result}")
    return result


# Augment the LLM with tools
tools = [add, multiply, divide]
tools_by_name = {tool.name: tool for tool in tools}
model_with_tools = llm.bind_tools(tools)

# Step 2: Define state

from langchain.messages import AnyMessage
from typing_extensions import TypedDict, Annotated
import operator


class MessagesState(TypedDict):
    messages: Annotated[list[AnyMessage], operator.add]
    llm_calls: int

# Step 3: Define model node
from langchain.messages import SystemMessage


def llm_call(state: dict):
    """LLM decides whether to call a tool or not"""

    return {
        "messages": [
            model_with_tools.invoke(
                [
                    SystemMessage(
                        content="""You are an expert SAP BTP consultant with deep knowledge of cloud platforms and enterprise architecture.
                                    You can perform arithmetic calculations when needed using the provided tools.
                                    When answering technical questions, be precise and reference official SAP terminology.
                                 """
                    )
                ]
                + state["messages"]
            )
        ],
        "llm_calls": state.get('llm_calls', 0) + 1
    }


# Step 4: Define tool node

from langchain.messages import ToolMessage


def tool_node(state: dict):
    """Performs the tool call"""

    result = []
    for tool_call in state["messages"][-1].tool_calls:
        tool = tools_by_name[tool_call["name"]]
        observation = tool.invoke(tool_call["args"])
        result.append(ToolMessage(content=observation, tool_call_id=tool_call["id"]))
    return {"messages": result}

# Step 5: Define logic to determine whether to end

from typing import Literal
from langgraph.graph import StateGraph, START, END


# Conditional edge function to route to the tool node or end based upon whether the LLM made a tool call
def should_continue(state: MessagesState) -> Literal["tool_node", END]:
    """Decide if we should continue the loop or stop based upon whether the LLM made a tool call"""

    messages = state["messages"]
    last_message = messages[-1]

    # If the LLM makes a tool call, then perform an action
    if last_message.tool_calls:
        return "tool_node"

    # Otherwise, we stop (reply to the user)
    return END

# Step 6: Build agent

# Build workflow
agent_builder = StateGraph(MessagesState)

# Add nodes
agent_builder.add_node("llm_call", llm_call)
agent_builder.add_node("tool_node", tool_node)

# Add edges to connect nodes
agent_builder.add_edge(START, "llm_call")
agent_builder.add_conditional_edges(
    "llm_call",
    should_continue,
    ["tool_node", END]
)
agent_builder.add_edge("tool_node", "llm_call")

# Compile the agent
agent = agent_builder.compile()

# Test with clearer question
from langchain.messages import HumanMessage

print("=" * 60)
print("🤖 TESTING AGENT WITH DEBUG OUTPUT")
print("=" * 60)

# Test 1: Original question
print("\n📝 Question: Add 3 and 4 and 5")
print("-" * 60)
messages = agent.invoke({"messages": [HumanMessage(content="Add 3 and 4 and 5")]})
print(f"\n✅ Final Answer: {messages['messages'][-1].content}")

# Test 2: Clearer question
print("\n" + "=" * 60)
print("📝 Question: Calculate (3 + 4 + 5)")
print("-" * 60)
messages = agent.invoke({"messages": [HumanMessage(content="Calculate (3 + 4 + 5)")]})
print(f"\n✅ Final Answer: {messages['messages'][-1].content}")

print("\n" + "=" * 60)

🤖 TESTING AGENT WITH DEBUG OUTPUT

📝 Question: Add 3 and 4 and 5
------------------------------------------------------------
  🔧 add(3, 7) = 10

✅ Final Answer: The sum of 3, 4, and 5 is 12.

📝 Question: Calculate (3 + 4 + 5)
------------------------------------------------------------
  🔧 add(3, 4) = 7
  🔧 add(1, 5) = 6

✅ Final Answer: The result of the calculation (3 + 4 + 5) is: 12.

